In [1]:
# Dependencies
from bs4 import BeautifulSoup
from bs4.element import Comment
import requests
import csv
import re
from time import sleep
from random import randint
import json

In [2]:
#Build URL of the navigation pages - this page will contain hyperlinks to actual cases.
#e.g. if you put in 2016, it will have a page with SCC cases related to 2016. We will take those hyperlinks
#then later use those to scrape the actual cases
def buildurl(hyperlink):
    url = 'https://scc-csc.lexum.com'+str(hyperlink)+'?iframe=true'
    return url

def builduserurl(hyperlink):
    url_user = 'https://scc-csc.lexum.com'+str(hyperlink)
    return url_user

def print_to_txt(curr_case_code,soup_object):
    """Takes information and creates a file with the case text inside"""

    filename = curr_case_code+".txt"
    filepath = "Data/tax_cases/"+str(filename)

    text_to_print=""
    
    print("curr_case_code: "+curr_case_code)
    with open (filepath,'a') as file_object:
        for item in soup_object.body.findAll("p"):
            item = item.text
            item = item.rstrip()
            try:
                file_object.write(str(item))
                file_object.write(" ")
            except:
                pass

In [5]:
cases_folder_name = "Data/"
cases_file_name = str("tax_cases_info.txt")

#load list of cases already scraped as case_dict_list. Loading it with the try/except so itll work if first time running this,
#or second/third (after 50 requests have to fill in a captcha)
try:
    with open(str(cases_folder_name+cases_file_name)) as json_file:
            case_dict_list =  json.load(json_file)
except:
    case_dict_list=[]

len_cases_scraped = len(case_dict_list)

#load list of cases to scrape as tax_hyperlinks
with open(str("Data/hyperlinks/tax_hyperlinks.txt")) as json_file:
    hyperlinks =  json.load(json_file)


for i in range(0,51):
    #make URL then get response
    ###~~~~~FUNNEL HYPERLINK LIST IN HERE
    hyperlink=hyperlinks[len_cases_scraped+i]
    url = buildurl(hyperlink)
    url_user = builduserurl(hyperlink)
    print(url_user)
    ###~~~~~

    response = requests.get(url)
    
    print(response.status_code)
    if response.status_code == 429:
        print("429 Error")
        break
    
    if response.status_code == 403:
        print("403 Error")
        break
    
    #make soup object
    soup_object = BeautifulSoup(response.text, 'html.parser')

    #collect table stats
    case_table = soup_object.body.find("table")
    table_rows = case_table.findAll("tr")
    info_sub_dict={}
    for row in table_rows:
        columns = row.findAll("td")
        row_data=[]
        keys=[]
        values=[]
        for column in columns:
            row_data.append(column.text)

        info_sub_dict.update({row_data[0]:row_data[1].strip()})

    #get case_code
    try:
        case_code = "tax"+info_sub_dict["Report"]
    except:
        try:
            case_code = "tax"+str(info_sub_dict["Recueil"])+" FRENCH"
        except:
            case_code="tax"+info_sub_dict["Case number"]

    
    info_sub_dict.update({"case_code":case_code})
        
    #get case_name
    case_name = soup_object.body.find("h3", class_="title").text
    info_sub_dict.update({"case_name":case_name})
        
    #build user url

    info_sub_dict.update({"url":url_user})
    info_sub_dict.update({"crim_tax":"tax"})

    #update info_dict_list
    case_dict_list.append(info_sub_dict)

    #print case contents to file
    print_to_txt(case_code,soup_object)

    print("~~")
    sleep(0.5)
    #print info_dict_list to file
    with open(str(cases_folder_name+cases_file_name), 'w') as outfile:
        json.dump(case_dict_list, outfile)

https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1592/index.do
200
curr_case_code: tax[1998] 1 SCR 196
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1591/index.do
200
curr_case_code: tax[1998] 1 SCR 183
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1524/index.do
200
curr_case_code: tax[1997] 2 SCR 336
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1382/index.do
200
curr_case_code: tax[1996] 2 SCR 161
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1375/index.do
200
curr_case_code: tax[1996] 1 SCR 963
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1346/index.do
200
curr_case_code: tax[1996] 1 SCR 254
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1347/index.do
200
curr_case_code: tax[1996] 1 SCR 305
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1338/index.do
200
curr_case_code: tax[1996] 1 SCR 73
~~
https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/1288/index.do
200
curr_case_code: tax[1995] 3 SCR 103
~~
https://scc-csc.lexum.com/scc-csc/scc-

IndexError: list index out of range